In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
# Load the dataset
file_path = "../data/Alzheimer_s_Disease_and_Healthy_Aging_Indicators__Cognitive_Decline_20250131.csv"
df = pd.read_csv(file_path)

# display the first few rows
df.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,Datasource,Class,Topic,Question,Data_Value_Unit,DataValueTypeID,...,Stratification2,Geolocation,ClassID,TopicID,QuestionID,LocationID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2
0,2022,2022,AZ,Arizona,BRFSS,Cognitive Decline,Functional difficulties associated with subjec...,Percentage of older adults who reported subjec...,%,PRCTG,...,Female,POINT (-111.7638113 34.86597028),C06,TCC02,Q31,4,AGE,AGE_OVERALL,GENDER,FEMALE
1,2022,2022,AZ,Arizona,BRFSS,Cognitive Decline,Functional difficulties associated with subjec...,Percentage of older adults who reported subjec...,%,PRCTG,...,Hispanic,POINT (-111.7638113 34.86597028),C06,TCC02,Q31,4,AGE,5064,RACE,HIS
2,2022,2022,AZ,Arizona,BRFSS,Cognitive Decline,Functional difficulties associated with subjec...,Percentage of older adults who reported subjec...,%,PRCTG,...,"White, non-Hispanic",POINT (-111.7638113 34.86597028),C06,TCC02,Q31,4,AGE,65PLUS,RACE,WHT
3,2022,2022,AZ,Arizona,BRFSS,Cognitive Decline,Functional difficulties associated with subjec...,Percentage of older adults who reported subjec...,%,PRCTG,...,Native Am/Alaskan Native,POINT (-111.7638113 34.86597028),C06,TCC02,Q31,4,AGE,65PLUS,RACE,NAA
4,2022,2022,AZ,Arizona,BRFSS,Cognitive Decline,Functional difficulties associated with subjec...,Percentage of older adults who reported subjec...,%,PRCTG,...,"Black, non-Hispanic",POINT (-111.7638113 34.86597028),C06,TCC02,Q31,4,AGE,AGE_OVERALL,RACE,BLK


In [3]:
# get general information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22182 entries, 0 to 22181
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   YearStart                   22182 non-null  int64  
 1   YearEnd                     22182 non-null  int64  
 2   LocationAbbr                22182 non-null  object 
 3   LocationDesc                22182 non-null  object 
 4   Datasource                  22182 non-null  object 
 5   Class                       22182 non-null  object 
 6   Topic                       22182 non-null  object 
 7   Question                    22182 non-null  object 
 8   Data_Value_Unit             22182 non-null  object 
 9   DataValueTypeID             22182 non-null  object 
 10  Data_Value_Type             22182 non-null  object 
 11  Data_Value                  14136 non-null  float64
 12  Data_Value_Alt              14136 non-null  float64
 13  Data_Value_Footnote_Symbol  121

### Cleaning the dataset

In [4]:
# create a new "Year" column using the average of YearStart and YearEnd, round to the nearest integer
df['Year'] = ((df['YearStart'] + df['YearEnd']) / 2).round().astype(int)

In [5]:
# move the "Year" column to the front
col_order = ["Year"] + [col for col in df.columns if col != "Year"]
df = df[col_order]

In [6]:
# rename "LocationDesc" to "Location"
df.rename(columns={"LocationDesc" : "Location",
                  "Data_Value" : "Percentage_Value"}, 
          inplace=True)

# move "Location" column next to "Year"
col_order = ["Year", "Location"] + [col for col in df.columns if col not in ["Year", "Location"]]
df = df[col_order]

In [7]:
# modifying the topic column
# Ensure 'Topic' exists before proceeding
if 'Topic' in df.columns:
    # Define shortened names for the topics
    topic_mapping = {
        'Functional difficulties associated with subjective cognitive decline or memory loss among older adults': 'Functional_Difficulties',
        'Need assistance with day-to-day activities because of subjective cognitive decline or memory loss': 'Needs_Assistance',
        'Subjective cognitive decline or memory loss among older adults': 'Cognitive_Decline',
        'Talked with health care professional about subjective cognitive decline or memory loss': 'Consulted_Professional'
    }

    # Create new boolean columns for each topic
    for full_name, short_name in topic_mapping.items():
        df[short_name] = (df['Topic'] == full_name).astype(int)

In [8]:
# modifying the question column
# Define shortened names for the questions
question_mapping = {
    'Percentage of older adults who reported subjective cognitive decline or memory loss that interferes with their ability to engage in social activities or household chores': 'q_Interferes_Activities',
    'Percentage of older adults who reported that as a result of subjective cognitive decline or memory loss that they need assistance with day-to-day activities': 'q_Needs_Assistance',
    'Percentage of older adults who reported subjective cognitive decline or memory loss that is happening more often or is getting worse in the preceding 12 months': 'q_Worsening_Decline',
    'Percentage of older adults with subjective cognitive decline or memory loss who reported talking with a health care professional about it': 'q_Consulted_Professional'
}

# Create new boolean columns for each question
for full_name, short_name in question_mapping.items():
    df[short_name] = (df['Question'] == full_name).astype(int)

**New Columns**:
- `Small_Sample_Size`: 1 if sample size is <50 or relative standard error >30%, else 0.
- `No_Data_Available`: 1 if **no data is available**, else 0.
- `Regional_Issue`: 1 if **regional estimates do not represent all states**, else 0.
- `Few_States_Reported`: 1 if **fewer than 50 states reported**, else 0.

In [9]:
# Define mapping for boolean columns
df['Small_Sample_Size'] = df['Data_Value_Footnote'].str.contains("Sample size", na=False).astype(int)
df['No_Data_Available'] = df['Data_Value_Footnote'].str.contains("No Data Available", na=False).astype(int)
df['Regional_Issue'] = df['Data_Value_Footnote'].str.contains("Regional estimates", na=False).astype(int)
df['Few_States_Reported'] = df['Data_Value_Footnote'].str.contains("Fewer than 50 States", na=False).astype(int)

In [10]:
# Create new columns for Age Group and Race/Ethnicity
df['Age_Group'] = df.apply(lambda row: row['Stratification1'] if row['StratificationCategory1'] == "Age Group" else None, axis=1)
df['Race_Ethnicity'] = df.apply(lambda row: row['Stratification2'] if row['StratificationCategory2'] == "Race/Ethnicity" else None, axis=1)

# Fill missing values with appropriate defaults
df['Age_Group'].fillna("Overall", inplace=True)
df['Race_Ethnicity'].fillna("Not Specified", inplace=True)

/var/folders/vz/t7hhp_d94_x5_9qpjz3_2bf80000gn/T/ipykernel_44574/454108836.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age_Group'].fillna("Overall", inplace=True)
/var/folders/vz/t7hhp_d94_x5_9qpjz3_2bf80000gn/T/ipykernel_44574/454108836.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

In [11]:
# drop the unnecessary columns
df.drop(columns=["YearStart", "YearEnd", "LocationAbbr",
                "Datasource", "Class", "Data_Value_Unit",
                "DataValueTypeID", "Data_Value_Type",
                "Data_Value_Alt", "Data_Value_Footnote_Symbol",
                "Topic", "Question", "Data_Value_Footnote", "Geolocation",
                "ClassID", "TopicID", "QuestionID", "LocationID",
                "StratificationCategory1", "Stratification1", "StratificationCategory2", 
                "Stratification2", "StratificationCategoryID1", "StratificationID1",
                "StratificationCategoryID2", "StratificationID2"], 
        inplace=True)

In [12]:
df.head()

,Year,Location,Percentage_Value,Low_Confidence_Limit,High_Confidence_Limit,Functional_Difficulties,Needs_Assistance,Cognitive_Decline,Consulted_Professional,q_Interferes_Activities,q_Needs_Assistance,q_Worsening_Decline,q_Consulted_Professional,Small_Sample_Size,No_Data_Available,Regional_Issue,Few_States_Reported,Age_Group,Race_Ethnicity
0,2022,Arizona,31.6,23.5,41.0,1,0,0,0,1,0,0,0,0,0,0,0,Overall,Not Specified
1,2022,Arizona,NaN,NaN,NaN,1,0,0,0,1,0,0,0,1,0,0,0,50-64 years,Hispanic
2,2022,Arizona,19.9,14.0,27.3,1,0,0,0,1,0,0,0,0,0,0,0,65 years or older,"White, non-Hispanic"
3,2022,Arizona,NaN,NaN,NaN,1,0,0,0,1,0,0,0,1,0,0,0,65 years or older,Native Am/Alaskan Native
4,2022,Arizona,NaN,NaN,NaN,1,0,0,0,1,0,0,0,1,0,0,0,Overall,"Black, non-Hispanic"


### Filling the missing values with the median

In [13]:
df['Percentage_Value'].fillna(df['Percentage_Value'].median(), inplace=True)
df['Low_Confidence_Limit'].fillna(df['Low_Confidence_Limit'].median(), inplace=True)
df['High_Confidence_Limit'].fillna(df['High_Confidence_Limit'].median(), inplace=True)

/var/folders/vz/t7hhp_d94_x5_9qpjz3_2bf80000gn/T/ipykernel_44574/2873957093.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Percentage_Value'].fillna(df['Percentage_Value'].median(), inplace=True)
/var/folders/vz/t7hhp_d94_x5_9qpjz3_2bf80000gn/T/ipykernel_44574/2873957093.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whic

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22182 entries, 0 to 22181
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Year                      22182 non-null  int64  
 1   Location                  22182 non-null  object 
 2   Percentage_Value          22182 non-null  float64
 3   Low_Confidence_Limit      22182 non-null  float64
 4   High_Confidence_Limit     22182 non-null  float64
 5   Functional_Difficulties   22182 non-null  int64  
 6   Needs_Assistance          22182 non-null  int64  
 7   Cognitive_Decline         22182 non-null  int64  
 8   Consulted_Professional    22182 non-null  int64  
 9   q_Interferes_Activities   22182 non-null  int64  
 10  q_Needs_Assistance        22182 non-null  int64  
 11  q_Worsening_Decline       22182 non-null  int64  
 12  q_Consulted_Professional  22182 non-null  int64  
 13  Small_Sample_Size         22182 non-null  int64  
 14  No_Dat

### Saving the dataset

In [15]:
# save the cleaned dataset to a CSV file
cleaned_file_path = "../data/cleaned_cognitive_decline.csv"
df.to_csv(cleaned_file_path, index=False)